# Widget Example Using Ginga

See https://astrowidgets.readthedocs.io for additional details about the widget, including installation notes.

In [ ]:
from astrowidgets import ImageWidget

In [ ]:
from ginga.misc.log import get_logger

logger = get_logger('my viewer', log_stderr=True,
                    log_file=None, level=30)

In [ ]:
w = ImageWidget(logger=logger)

For this example, we use an image from Astropy data repository and load it as `CCDData`. Feel free to modify `filename` to point to your desired image.

Alternately, for local FITS file, you could load it like this instead:
```python
w.load_fits(filename, numhdu=numhdu)
```    
Or if you wish to load a data array natively (without WCS):
```python
from astropy.io import fits
with fits.open(filename, memmap=False) as pf:
    arr = pf[numhdu].data.copy()
w.load_array(arr)
```

In [ ]:
filename = 'http://data.astropy.org/photometry/spitzer_example_image.fits'
numhdu = 0

# Loads NDData
# NOTE: memmap=False is needed for remote data on Windows.
# NOTE: Some file also requires unit to be explicitly set in CCDData.
from astropy.nddata import CCDData
ccd = CCDData.read(filename, hdu=numhdu, format='fits')
w.load_nddata(ccd)

Ginga key bindings documented at http://ginga.readthedocs.io/en/latest/quickref.html . Note that not all documented bindings would work here. Please use an alternate binding, if available, if the chosen one is not working.

Here are the ones that worked during testing with Firefox 52.8.0 on RHEL7 64-bit:

Key | Action | Notes
--- | --- | ---
`+` | Zoom in |
`-` | Zoom out |
Number (0-9) | Zoom in to specified level | 0 = 10
Shift + number | Zoom out to specified level | Numpad does not work
` (backtick) | Reset zoom |
Space > `q` > arrow | Pan |
ESC | Exit mode (pan, etc) |
`c` | Center image
Space > `d` > up/down arrow | Cycle through color distributions
Space > `d` > Shift + `d` | Go back to linear color distribution
Space > `s` > Shift + `s` | Set cut level to min/max
Space > `s` > Shift + `a` | Set cut level to 0/255 (for 8bpp RGB images)
Space > `s` > up/down arrow | Cycle through cuts algorithms
Space > `l` | Toggle no/soft/normal lock |

*TODO: Check out Contrast Mode next*

A viewer will be shown after running the next cell.
In Jupyter Lab, you can split it out into a separate view by right-clicking on the viewer and then select
"Create New View for Output". Then, you can drag the new
"Output View" tab, say, to the right side of the workspace. Both viewers are connected to the same events.

In [ ]:
w

This next cell captures print outputs. You can pop it out like the viewer above. It is very convenient for debugging purpose.

In [ ]:
# Capture print outputs from the widget
display(w.print_out)

The following cell changes the visibility or position of the cursor info bar.


In [ ]:
w.cursor = 'top'  # 'top', 'bottom', None
print(w.cursor)

The rest of the calls demonstrate how the widget API works. Comment/uncomment as needed. Feel free to experiment.

In [ ]:
# Programmatically center to (X, Y) on viewer
w.center_on((1, 1))

In [ ]:
# Programmatically offset w.r.t. current center
w.offset_by(10, 10)

In [ ]:
from astropy.coordinates import SkyCoord

# Change the values here if you are not using given
# example image.
ra_str = '01h13m23.193s'
dec_str = '+00d12m32.19s'
frame = 'icrs'

# Programmatically center to SkyCoord on viewer
w.center_on(SkyCoord(ra_str, dec_str, frame=frame))

In [ ]:
from astropy import units as u

# Change the values if needed.
deg_offset = 0.001 * u.deg

# Programmatically offset (in degrees) w.r.t.
# SkyCoord center
w.offset_by(deg_offset, deg_offset)

In [ ]:
# Show zoom level
print(w.zoom_level)

In [ ]:
# Programmatically zoom image on viewer
w.zoom(2)

In [ ]:
# Capture what viewer is showing and save RGB image.
# Need https://github.com/ejeschke/ginga/pull/688 to work.
w.save('test.png')

In [ ]:
# Get all available image stretch options
print(w.stretch_options)

In [ ]:
# Get image stretch algorithm in use
print(w.stretch)

In [ ]:
# Change the stretch
w.stretch = 'histeq'
print(w.stretch)

In [ ]:
# Get all available image cuts options
print(w.autocut_options)

In [ ]:
# Get image cut levels in use
print(w.cuts)

In [ ]:
# Change the cuts by providing explicit low/high values
w.cuts = (0, 100)
print(w.cuts)

In [ ]:
# Change the cuts with an autocut algorithm
w.cuts = 'zscale'
print(w.cuts)

In [ ]:
# This enables click to center.
w.click_center = True

Now, click on the image to center it.

In [ ]:
# Turn it back off so marking (next cell) can be done.
w.click_center = False

In [ ]:
# This enables marking mode.
w.start_marking()
print(w.is_marking)

Now, click on the image to mark a point of interest.

In [ ]:
# When done, set back to False.
w.stop_marking()
print(w.is_marking)

In [ ]:
# Get table of markers
markers_table = w.get_markers(marker_name='all')

# Default display might be hard to read, so we do this
print('{:^8s} {:^8s} {:^28s}'.format(
    'X', 'Y', 'Coordinates'))
for row in markers_table:
    c = row['coord'].to_string('hmsdms')
    print('{:8.2f} {:8.2f} {}'.format(
        row['x'], row['y'], c))

In [ ]:
# Erase markers from display
w.reset_markers()

The following works even when we have set `w.is_marking=False`. This is because `w.is_marking` only controls the interactive marking and does not affect marking programmatically.

In [ ]:
# Programmatically re-mark from table using X, Y.
# To be fancy, first 2 points marked as bigger
# and thicker red circles.
w.marker = {'type': 'circle', 'color': 'red', 'radius': 50,
            'linewidth': 2}
w.add_markers(markers_table[:2])
# You can also change the type of marker to cross or plus
w.marker = {'type': 'cross', 'color': 'cyan', 'radius': 20}
w.add_markers(markers_table[2:])

In [ ]:
# Erase them again
w.reset_markers()

# Programmatically re-mark from table using SkyCoord
w.add_markers(markers_table, use_skycoord=True)

In [ ]:
# Start marking again
w.start_marking()
print(w.is_marking)

In [ ]:
# Stop marking AND clear markers.
# Note that this deletes ALL of the markers
w.stop_marking(clear_markers=True)
print(w.is_marking)

The next cell randomly generates some "stars" to mark. In the real world, you would probably detect real stars using `photutils` package.

In [ ]:
import numpy as np
from astropy.table import Table

# Maximum umber of "stars" to generate randomly.
max_stars = 1000

# Number of pixels from edge to avoid.
dpix = 20

# Image from the viewer.
img = w._viewer.get_image()

# Random "stars" generated.
bad_locs = np.random.randint(
    dpix, high=img.shape[1] - dpix, size=[max_stars, 2])

# Only want those not near the edges.
mask = ((dpix < bad_locs[:, 0]) &
        (bad_locs[:, 0] < img.shape[0] - dpix) &
        (dpix < bad_locs[:, 1]) &
        (bad_locs[:, 1] < img.shape[1] - dpix))
locs = bad_locs[mask]

# Put them in table
t = Table([locs[:, 1], locs[:, 0]], names=('x', 'y'))
print(t)

In [ ]:
# Mark those "stars" based on given table with X and Y.
w.add_markers(t)

The following illustrates how to control number of markers displayed using interactive widget from `ipywidgets`.

In [ ]:
# Set the marker properties as you like.
w.marker = {'type': 'circle', 'color': 'red', 'radius': 10,
            'linewidth': 2}

# Define a function to control marker display
def show_circles(n):
    """Show and hide circles."""
    w.reset_markers()
    t2show = t[:n]
    w.add_markers(t2show)
    with w.print_out:
        print('Displaying {} markers...'.format(len(t2show)))

We redisplay the image widget below above the slider. Note that the slider affects both this view of the image widget and the one near the top of the notebook.

In [ ]:
from IPython.display import display

import ipywidgets as ipyw
from ipywidgets import interactive

# Show the slider widget.
slider = interactive(show_circles,
                     n=ipyw.IntSlider(min=0,max=len(t),step=1,value=0, continuous_update=False))
display(w, slider)

Now, use the slider. The chosen `n` represents the first `n` "stars" being displayed.